# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from absa_models.ABSA_BERT_Dropout_BiLSTM_Linear import ABSA_BERT_Dropout_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ABSA_MAMS_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,absa_tags
0,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, ..."
1,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, ..."
2,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, ..."
3,"when tables opened up, the manager sat another...","[when, tables, opened, up, ,, the, manager, sa...","[0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
4,"when tables opened up, the manager sat another...","[when, tables, opened, up, ,, the, manager, sa...","[0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
MODEL_OUTPUT = '../../../results/ABSA/MAMS/models/bert_pre_trained_dropout_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/ABSA/MAMS/stats/bert_pre_trained_dropout_bilstm_linear.csv'

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
best_accuracy = 0.0

In [13]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [14]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 4), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [15]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [16]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [17]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ABSA_BERT_Dropout_BiLSTM_Linear(BertModel.from_pretrained('bert-base-uncased'), dropout=0.3, bilstm_in_features=256, no_out_labels=4, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/2237, Loss: 1.3831692934036255
Epoch: 0/2, Batch: 223/2237, Loss: 0.30869072675704956
Epoch: 0/2, Batch: 446/2237, Loss: 0.17631593346595764
Epoch: 0/2, Batch: 669/2237, Loss: 0.43559253215789795
Epoch: 0/2, Batch: 892/2237, Loss: 0.2408280372619629
Epoch: 0/2, Batch: 1115/2237, Loss: 0.15919789671897888
Epoch: 0/2, Batch: 1338/2237, Loss: 0.2964566946029663
Epoch: 0/2, Batch: 1561/2237, Loss: 0.2248377799987793
Epoch: 0/2, Batch: 1784/2237, Loss: 0.15100184082984924
Epoch: 0/2, Batch: 2007/2237, Loss: 0.06231212615966797
Epoch: 0/2, Batch: 2230/2237, Loss: 0.14944525063037872
Epoch: 1/2, Batch: 0/2237, Loss: 0.14583805203437805
Epoch: 1/2, Batch: 223/2237, Loss: 0.08851128816604614
Epoch: 1/2, Batch: 446/2237, Loss: 0.11292742192745209
Epoch: 1/2, Batch: 669/2237, Loss: 0.053483009338378906
Epoch: 1/2, Batch: 892/2237, Loss: 0.13002857565879822
Epoch: 1/2, Batch: 1115/2237, Loss: 0.128607839345932
Epoch: 1/2, Batch: 1338/2237, Loss: 0.24498945474624634
Ep

In [18]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.969338,0.969338,0.843482,0.969338,0.906365,0.969338,0.872969,499.511999
1,0.965525,0.965525,0.812255,0.965525,0.906358,0.965525,0.852922,492.238224
2,0.969544,0.969544,0.853579,0.969544,0.893324,0.969544,0.872303,497.431457
3,0.967291,0.967291,0.838480,0.967291,0.901756,0.967291,0.867565,498.653585
4,0.967573,0.967573,0.835216,0.967573,0.906720,0.967573,0.868560,487.667144
5,0.970788,0.970788,0.857319,0.970788,0.922418,0.970788,0.888012,440.269203
6,0.965710,0.965710,0.828253,0.965710,0.900834,0.965710,0.861736,488.172769
7,0.962550,0.962550,0.809245,0.962550,0.906574,0.962550,0.853415,488.568213
8,0.969045,0.969045,0.841615,0.969045,0.903127,0.969045,0.870674,484.503408
9,0.969881,0.969881,0.839314,0.969881,0.919748,0.969881,0.876117,477.340663


In [19]:
results.to_csv(STATS_OUTPUT)